<a href="https://colab.research.google.com/github/allexanderprastya/PROJECT-TA-JURNAL/blob/main/labelling_lexicon_base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pelabelan Data Menggunakan Lexicon

In [2]:
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 7.9 MB/s 


In [8]:
import pandas as pd
import re
import string
import numpy as np
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [43]:
df = pd.read_csv('/content/review_15k_clean.csv')
df

,index,content,sentiment
0,0,bangke gw turun nyangkut kenok langsung tolong...,positive
1,1,manga pa pubg selalu loding ketika login padah...,negative
2,2,bag nya dong developer perbaiki semua ngeluh k...,positive
3,3,emang berat putaran kenapa gak seperti biasany...,neutral
4,5,game ini sudah bagus tapi sayang ping nya sela...,positive
...,...,...,...
14738,14995,game ini sangat burik dan sangat hd dengan keb...,positive
14739,14996,lemot,neutral
14740,14997,bagus banget,positive
14741,14998,game seru keren hd tapi banyak bug pemakaian m...,neutral


In [45]:
df.drop('index',axis=1, inplace=True)

In [46]:
df

,content,sentiment
0,bangke gw turun nyangkut kenok langsung tolong...,positive
1,manga pa pubg selalu loding ketika login padah...,negative
2,bag nya dong developer perbaiki semua ngeluh k...,positive
3,emang berat putaran kenapa gak seperti biasany...,neutral
4,game ini sudah bagus tapi sayang ping nya sela...,positive
...,...,...
14738,game ini sangat burik dan sangat hd dengan keb...,positive
14739,lemot,neutral
14740,bagus banget,positive
14741,game seru keren hd tapi banyak bug pemakaian m...,neutral


In [47]:
word_dict = {}
for i in range(0,len(df['content'])):
    sentence = df['content'][i]
    word_token = word_tokenize(sentence)
    for j in word_token:
        if j not in word_dict:
            word_dict[j] = 1
        else:
            word_dict[j] += 1

In [48]:
len(word_dict)

10685

In [49]:
len({k:v for (k,v) in word_dict.items() if v < 4})

7084

In [50]:
negasi = ['bukan','tidak','ga','gk']
lexicon = pd.read_csv('/content/modified_full_lexicon.csv')
lexicon = lexicon.drop(lexicon[(lexicon['word'] == 'bukan')
                               |(lexicon['word'] == 'tidak')
                               |(lexicon['word'] == 'ga')
                               |(lexicon['word'] == 'gk') ].index,axis=0)
lexicon = lexicon.reset_index(drop=True)

In [51]:
len(lexicon)

10248

In [52]:
lexicon.head(10)

,word,weight,number_of_words
0,hai,3,1
1,merekam,2,1
2,ekstensif,3,1
3,paripurna,1,1
4,detail,2,1
5,pernik,3,1
6,belas,2,1
7,welas,4,1
8,kabung,1,1
9,rahayu,4,1


In [53]:
lexicon_word = lexicon['word'].to_list()
lexicon_num_words = lexicon['number_of_words']

In [54]:
len(lexicon_word)

10248

In [55]:
ns_words = []
factory = StemmerFactory()
stemmer = factory.create_stemmer()
for word in word_dict.keys():
    if word not in lexicon_word:
        kata_dasar = stemmer.stem(word)
        if kata_dasar not in lexicon_word:
            ns_words.append(word)
len(ns_words)

8001

In [56]:
len({k:v for (k,v) in word_dict.items() if ((k in ns_words)&(v>3)) })

2250

In [57]:
ns_words_list = {k:v for (k,v) in word_dict.items() if ((k in ns_words)&(v>3))}

In [58]:
sort_orders = sorted(ns_words_list.items(), key=lambda x: x[1], reverse=True)
sort_orders=sort_orders[0:20]
for i in sort_orders:
    print(i[0], i[1])

di 4661
nya 3740
bug 2613
bisa 2437
ini 2389
pubg 2383
dan 2302
tapi 1745
tidak 1583
update 1355
hp 1346
untuk 1262
yg 1256
gak 1244
sangat 1083
padahal 948
kenapa 872
ga 849
dulu 820
udah 813


In [59]:
lexicon['number_of_words'].value_counts()

1    9536
2     686
3      24
4       2
Name: number_of_words, dtype: int64

In [60]:
sencol =[]
senrow =np.array([])
nsen = 0
factory = StemmerFactory()
stemmer = factory.create_stemmer()
sentiment_list = []
# function to write the word's sentiment if it is founded
def found_word(ind,words,word,sen,sencol,sentiment,add):
    # if it is already included in the bag of words matrix, then just increase the value
    if word in sencol:
        sen[sencol.index(word)] += 1
    else:
    #if not, than add new word
        sencol.append(word)
        sen.append(1)
        add += 1
    #if there is a negation word before it, the sentiment would be the negation of it's sentiment
    if (words[ind-1] in negasi):
        sentiment += -lexicon['weight'][lexicon_word.index(word)]
    else:
        sentiment += lexicon['weight'][lexicon_word.index(word)]
    
    return sen,sencol,sentiment,add
            
# checking every words, if they are appear in the lexicon, and then calculate their sentiment if they do
for i in range(len(df)):
    nsen = senrow.shape[0]
    words = word_tokenize(df['content'][i])
    sentiment = 0 
    add = 0
    prev = [0 for ii in range(len(words))]
    n_words = len(words)
    if len(sencol)>0:
        sen =[0 for j in range(len(sencol))]
    else:
        sen =[]
    
    for word in words:
        ind = words.index(word)
        # check whether they are included in the lexicon
        if word in lexicon_word :
            sen,sencol,sentiment,add= found_word(ind,words,word,sen,sencol,sentiment,add)
        else:
        # if not, then check the root word
            kata_dasar = stemmer.stem(word)
            if kata_dasar in lexicon_word:
                sen,sencol,sentiment,add= found_word(ind,words,kata_dasar,sen,sencol,sentiment,add)
        # if still negative, try to match the combination of words with the adjacent words
            elif(n_words>1):
                if ind-1>-1:
                    back_1    = words[ind-1]+' '+word
                    if (back_1 in lexicon_word):
                        sen,sencol,sentiment,add= found_word(ind,words,back_1,sen,sencol,sentiment,add)
                    elif(ind-2>-1):
                        back_2    = words[ind-2]+' '+back_1
                        if back_2 in lexicon_word:
                            sen,sencol,sentiment,add= found_word(ind,words,back_2,sen,sencol,sentiment,add)
    # if there is new word founded, then expand the matrix
    if add>0:  
        if i>0:
            if (nsen==0):
                senrow = np.zeros([i,add],dtype=int)
            elif(i!=nsen):
                padding_h = np.zeros([nsen,add],dtype=int)
                senrow = np.hstack((senrow,padding_h))
                padding_v = np.zeros([(i-nsen),senrow.shape[1]],dtype=int)
                senrow = np.vstack((senrow,padding_v))
            else:
                padding =np.zeros([nsen,add],dtype=int)
                senrow = np.hstack((senrow,padding))
            senrow = np.vstack((senrow,sen))
        if i==0:
            senrow = np.array(sen).reshape(1,len(sen))
    # if there isn't then just update the old matrix
    elif(nsen>0):
        senrow = np.vstack((senrow,sen))
        
    sentiment_list.append(sentiment)

In [61]:
len(sentiment_list)

14743

In [62]:
print(senrow.shape[0])

14743


In [63]:
sencol.append('sentiment')
sentiment_array = np.array(sentiment_list).reshape(senrow.shape[0],1)
sentiment_data = np.hstack((senrow,sentiment_array))
df_sen = pd.DataFrame(sentiment_data,columns = sencol)

In [64]:
df_sen.head(10)

,turun,langsung,tolong,salah,apa,pa,kuota,beli,baik,semua,...,sir,fresh,jasa,eks,haloo,suplai,remaja,berlawanan,selidik,sentiment
0,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,-5
2,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,-4
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,7
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
5,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,7
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-2
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3


In [65]:
cek_df = pd.DataFrame([])
cek_df['text'] = df['content'].copy()
cek_df['sentiment']  = df_sen['sentiment'].copy()

In [66]:
cek_df.head(10)

,text,sentiment
0,bangke gw turun nyangkut kenok langsung tolong...,0
1,manga pa pubg selalu loding ketika login padah...,-5
2,bag nya dong developer perbaiki semua ngeluh k...,-4
3,emang berat putaran kenapa gak seperti biasany...,7
4,game ini sudah bagus tapi sayang ping nya sela...,10
5,makin di update makin lemot nih game aneh tolo...,7
6,agak gimana ya,4
7,udh gak bagus versi,2
8,udah dnwload hbis ga bisa di buka suruh donwlo...,-2
9,puas,3


In [67]:
cek_df['sentiment'] = df_sen['sentiment'].copy()
df.loc[cek_df['sentiment']>=0, 'sentiment'] = 1 #positive
df.loc[cek_df['sentiment']<0, 'sentiment'] = 0 #negative
df.loc[cek_df['sentiment']==0, 'sentiment'] = 2 #netral
#df.drop(df[cek_df.sentiment==0].index, inplace=True)
#df.reset_index()
df

,content,sentiment
0,bangke gw turun nyangkut kenok langsung tolong...,2
1,manga pa pubg selalu loding ketika login padah...,0
2,bag nya dong developer perbaiki semua ngeluh k...,0
3,emang berat putaran kenapa gak seperti biasany...,1
4,game ini sudah bagus tapi sayang ping nya sela...,1
...,...,...
14738,game ini sangat burik dan sangat hd dengan keb...,1
14739,lemot,0
14740,bagus banget,1
14741,game seru keren hd tapi banyak bug pemakaian m...,0


In [68]:
df['sentiment'].value_counts()

1    8313
0    4370
2    2060
Name: sentiment, dtype: int64

In [70]:
df.to_csv('lexiconDataLabelling.csv', encoding='utf-8', index=None, header=True)